<a href="https://colab.research.google.com/github/heleownae/DA_Insurance_PJT/blob/main/%ED%8C%8C%EC%83%9D%EB%B3%80%EC%88%98%EC%83%9D%EC%84%B1d%2Ce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
#라이브러리 임포트
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


plt.rc('font', family='NanumBarunGothic')

In [7]:
# !sudo apt-get install -y fonts-nanum
# !sudo fc-cache -fi
# !rm ~/. cache/matplotlib -rf

In [8]:
rlps = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/최종프/use_table/SDB2_I_CT_RLPS.csv')
cont = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/최종프/use_table/SDB2_I_CT_CONT_PVT.csv')
mgg = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/최종프/use_table/SDB2_I_CT_MGG_PVT.csv')
accd = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/최종프/use_table/SDB2_I_PT_ACCD.csv')
m_rlps = pd.read_csv('/rlps_fin3.csv')

d. 개인별 평균 청구 빈도 => 청구 테이블로 가야한다.
    1. (한 증권번호에 대해서) 지급이 됐든 안 됐든, 보험 청구를 얼마나 청구했는지
    2. 증권번호로 group by 후 청구일련번호 count
=> '청구사고 정보 테이블' (ACCD) 차주일련번호 별 청구일련번호 count

e. 개인이 가입한 보험계약 수

# 0번 전처리()

In [9]:
pre_0 = cont.groupby('POL_SN')[['YM_201806', 'YM_201807', 'YM_201808',
       'YM_201809', 'YM_201810', 'YM_201811', 'YM_201812', 'YM_201901',
       'YM_201902', 'YM_201903', 'YM_201904', 'YM_201905', 'YM_201906',
       'YM_201907', 'YM_201908', 'YM_201909', 'YM_201910', 'YM_201911',
       'YM_201912', 'YM_202001', 'YM_202002', 'YM_202003', 'YM_202004',
       'YM_202005', 'YM_202006']].sum().reset_index()
# pre_0 = cont.groupby('POL_SN')[['YM_202005','YM_202006']].sum().reset_index()

#이상치만 모으기
pre = pre_0[pre_0['YM_202006']>1]#13882개의 증권번호
display(pre)

#이상치 제거
cont = cont[~cont['POL_SN'].isin(pre['POL_SN'])]
display(cont)

,POL_SN,YM_201806,YM_201807,YM_201808,YM_201809,YM_201810,YM_201811,YM_201812,YM_201901,YM_201902,...,YM_201909,YM_201910,YM_201911,YM_201912,YM_202001,YM_202002,YM_202003,YM_202004,YM_202005,YM_202006
22,573,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
33,1146,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
52,1928,13,13,13,13,13,13,13,13,13,...,13,13,13,13,13,13,13,13,13,13
68,2578,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
84,3203,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244170,9995198,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
244185,9995730,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
244188,9995942,0,0,0,0,0,1,1,1,2,...,2,2,2,2,2,2,2,2,2,2
244197,9996385,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2


,SCTR_CD,POL_SN,IS_GRP_INS,INS_GBN_CD,INS_CT_STAT_CD,CT_CNCLS_DT,CT_TRMNT_DT,CT_ST_DT,CT_END_DT,CT_EFCTV_DT,...,YM_201909,YM_201910,YM_201911,YM_201912,YM_202001,YM_202002,YM_202003,YM_202004,YM_202005,YM_202006
0,6,3023157,0,3,7,200806,200903,200806,200906,999912,...,1,1,1,1,1,1,1,1,1,1
1,8,2067576,0,1,2,201012,999912,201012,999912,999912,...,1,1,1,1,1,1,1,1,1,1
2,6,7995898,0,7,2,201409,999912,201409,999912,999912,...,0,0,0,0,0,0,0,0,0,0
3,8,9743978,0,4,7,201604,201704,201604,201704,999912,...,1,1,1,1,1,1,1,1,1,1
4,6,9040681,0,4,2,201804,999912,201804,204804,999912,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368476,8,2330274,1,4,5,201905,999912,201905,202005,999912,...,0,0,0,0,0,0,0,0,0,1
368478,8,6242384,0,7,2,201212,999912,201212,203812,999912,...,1,1,1,1,1,1,1,1,1,1
368480,6,6050915,0,7,7,200902,201107,200902,201902,999912,...,1,1,1,1,1,1,1,1,1,1
368481,6,9901184,0,14,2,201809,999912,201809,201902,999912,...,0,0,0,0,0,0,0,0,0,0


In [7]:
accd.nunique()

JOIN_SN           17591
JOIN_SN_TYP           1
SCTR_CD               3
COM_SN            22189
IS_ME                 1
ACCD_SN           39650
REQ_SN            33603
IS_INDIV              1
GIS_CD               17
ACCD_DT              25
FST_RPT_DT           25
ACCD_GIS_CD          17
IN_DRV_RLTN_CD        6
dtype: int64

In [8]:
#175971명의 피보험자 차주에 대해 구한 보험금 청구건수
# REQ_SN 청구일련번호 고유개수 카운트
accd_count = accd.groupby('JOIN_SN')['REQ_SN'].nunique().reset_index()

In [10]:
accd_count

,JOIN_SN,REQ_SN
0,8,1
1,98,2
2,514,4
3,608,1
4,1064,5
...,...,...
17586,1958773,2
17587,1959050,4
17588,1959155,2
17589,1959315,1


In [11]:
accd_count.rename(columns = {'JOIN_SN' : '차주번호', 'REQ_SN' : '청구건수'}, inplace=True)
accd_count

accd_count.to_csv('인당청구건수.csv', index = False)

In [ ]:
#보험 계약정보에서 count하려고 했으나 차주번호가 없는 관계로
#보험 담보정보테이블에서 차주일련번호 별 고유증권번호를count 함


In [10]:
cont

,SCTR_CD,POL_SN,IS_GRP_INS,INS_GBN_CD,INS_CT_STAT_CD,CT_CNCLS_DT,CT_TRMNT_DT,CT_ST_DT,CT_END_DT,CT_EFCTV_DT,...,YM_201909,YM_201910,YM_201911,YM_201912,YM_202001,YM_202002,YM_202003,YM_202004,YM_202005,YM_202006
0,6,3023157,0,3,7,200806,200903,200806,200906,999912,...,1,1,1,1,1,1,1,1,1,1
1,8,2067576,0,1,2,201012,999912,201012,999912,999912,...,1,1,1,1,1,1,1,1,1,1
2,6,7995898,0,7,2,201409,999912,201409,999912,999912,...,0,0,0,0,0,0,0,0,0,0
3,8,9743978,0,4,7,201604,201704,201604,201704,999912,...,1,1,1,1,1,1,1,1,1,1
4,6,9040681,0,4,2,201804,999912,201804,204804,999912,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368476,8,2330274,1,4,5,201905,999912,201905,202005,999912,...,0,0,0,0,0,0,0,0,0,1
368478,8,6242384,0,7,2,201212,999912,201212,203812,999912,...,1,1,1,1,1,1,1,1,1,1
368480,6,6050915,0,7,7,200902,201107,200902,201902,999912,...,1,1,1,1,1,1,1,1,1,1
368481,6,9901184,0,14,2,201809,999912,201809,201902,999912,...,0,0,0,0,0,0,0,0,0,0


In [11]:
m = pd.merge(m_rlps[['JOIN_SN','POL_SN']], cont, how='left', on='POL_SN')

In [51]:
m

,JOIN_SN,POL_SN,SCTR_CD,IS_GRP_INS,INS_GBN_CD,INS_CT_STAT_CD,CT_CNCLS_DT,CT_TRMNT_DT,CT_ST_DT,CT_END_DT,...,YM_201909,YM_201910,YM_201911,YM_201912,YM_202001,YM_202002,YM_202003,YM_202004,YM_202005,YM_202006
0,8,795034,3.0,0.0,1.0,2.0,201601.0,999912.0,201601.0,209001.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,8,795034,3.0,0.0,1.0,7.0,201601.0,201903.0,201601.0,209001.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,8,3700186,3.0,0.0,4.0,7.0,201110.0,201903.0,201110.0,206910.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,8,3700186,3.0,0.0,4.0,2.0,201110.0,999912.0,201110.0,206910.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,8,6394418,6.0,0.0,3.0,2.0,201110.0,999912.0,201110.0,208910.0,...,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253683,1959698,7435599,6.0,0.0,13.0,7.0,201402.0,201905.0,201402.0,203402.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
253684,1959698,8764404,6.0,0.0,5.0,2.0,200803.0,999912.0,200803.0,202803.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
253685,1959919,2748732,6.0,1.0,4.0,5.0,200705.0,201005.0,200705.0,201005.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
253686,1959919,3831658,3.0,0.0,1.0,1.0,201904.0,999912.0,201904.0,999912.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
cont[cont['POL_SN']==111]

,SCTR_CD,POL_SN,IS_GRP_INS,INS_GBN_CD,INS_CT_STAT_CD,CT_CNCLS_DT,CT_TRMNT_DT,CT_ST_DT,CT_END_DT,CT_EFCTV_DT,...,YM_201909,YM_201910,YM_201911,YM_201912,YM_202001,YM_202002,YM_202003,YM_202004,YM_202005,YM_202006
327314,8,111,0,1,2,201002,999912,201002,999912,999912,...,1,1,1,1,1,1,1,1,1,1
